# Clustering Crypto

In [36]:
# Initial imports
import pandas as pd
import hvplot.pandas
# from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [37]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
#load the data
file_path = "C:/Users/butte/Classwork/Cryptocurrencies/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [38]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
new_crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
print(new_crypto_df.shape)
new_crypto_df

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [39]:
# Keep all the cryptocurrencies that have a working algorithm.
# "Check if the Algorithm column has any null values."
# YOUR CODE HERE
for column in new_crypto_df.columns:
    print(f"Column {column} has {new_crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [40]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
new_crypto_df = new_crypto_df.drop(['IsTrading'], axis = 1)
print(new_crypto_df.shape)
new_crypto_df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [41]:
# new_crypto_df = new_crypto_df.dropna(axis = 0, how = "any")
new_crypto_df = new_crypto_df.dropna(axis = 0)
print(new_crypto_df.shape)
new_crypto_df.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [42]:
# Keep the rows where coins are mined.
# YOUR CODE HERE
new_crypto_df = new_crypto_df[new_crypto_df["TotalCoinsMined"]> 0]
new_crypto_df.shape

(532, 5)

In [43]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
coin_name_df = new_crypto_df.drop(['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], axis = 1)
print(coin_name_df.shape)
coin_name_df.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [44]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
cluster_df = new_crypto_df.drop(['CoinName'], axis = 1)
print(cluster_df.shape)
cluster_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [45]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
cluster_encoded = pd.get_dummies(cluster_df, columns = ["Algorithm", "ProofType"])
cluster_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
cluster_scaled = StandardScaler().fit_transform(cluster_encoded)
print(cluster_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [47]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
pca = PCA(n_components = 3)
crypto_pca = pca.fit_transform(cluster_scaled)
crypto_pca

array([[-0.34272759,  0.98632301, -0.55990678],
       [-0.32600016,  0.98723518, -0.56034124],
       [ 2.32509019,  1.82234698, -0.63119258],
       ...,
       [ 0.34958227, -2.09807299,  0.38137627],
       [-0.12754853, -1.8306499 ,  0.42438509],
       [-0.30616236,  0.65260446, -0.25394986]])

In [48]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
pcs_df = pd.DataFrame(
    data = crypto_pca, columns = ["PC 1", "PC 2", "PC 3"], index = cluster_df.index
)

pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.342728,0.986323,-0.559907
404,-0.326000,0.987235,-0.560341
1337,2.325090,1.822347,-0.631193
BTC,-0.141188,-1.309046,0.159898
ETH,-0.149689,-2.008460,0.350534


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [49]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1,11))

# loop throug K values and find the inertia
# Looking for the best k

for i in k:
    km = KMeans(n_clusters = i, random_state = 0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# create a dataframe to store the k values and their approprite inertia values.
# plot the elbow curve using hvplot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve", xticks = k)

C:\Users\butte\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [50]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
# YOUR CODE HERE
model.fit(pcs_df)

# Predict clusters
# YOUR CODE HERE
predictions = model.predict(pcs_df)

print(predictions)

[1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1
 1 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0
 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0
 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1
 1 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1
 0 0 1 0 0 0 0 1 3 3 0 0 0 1 3 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0
 1 1 1 1 0 1 1 0 1 1 0 3 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 0 0 0 1 1 1 0 1 0 1 0 1 

In [51]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = cluster_df.join(pcs_df, how = "inner")
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df["CoinName"] = coin_name_df["CoinName"]
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df["Class"] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.342728,0.986323,-0.559907,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326000,0.987235,-0.560341,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.325090,1.822347,-0.631193,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141188,-1.309046,0.159898,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.149689,-2.008460,0.350534,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166850,-1.109425,-0.029685,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387691,1.326785,-0.437983,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.152814,-2.158831,0.395302,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.148122,-2.008468,0.350508,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.127547,-1.830650,0.424385,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [53]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    z="PC 1",
    y="PC 2",
    x="PC 3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [54]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_df.drop(columns = ["PC 1", "PC 2", "PC 3"]).hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class]

In [55]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
len(clustered_df)

532

In [58]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
cluster_scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])
print(cluster_scaled[0:5])

[[0.00000000e+00 4.20000000e-11]
 [1.06585544e-03 5.32000000e-04]
 [2.95755135e-02 3.14159265e-01]
 [1.81084216e-05 2.10000000e-05]
 [1.08773140e-04 0.00000000e+00]]


In [64]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df = pd.DataFrame(cluster_scaled, columns = ["TotalCoinsMined", "TotalCoinSupply"], index = clustered_df.index)
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df["CoinName"] = clustered_df["CoinName"]
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df["Class"] = clustered_df["Class"]
plot_df.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,0.000000,4.200000e-11,42 Coin,1
404,0.001066,5.320000e-04,404Coin,1
1337,0.029576,3.141593e-01,EliteCoin,1
BTC,0.000018,2.100000e-05,Bitcoin,0
ETH,0.000109,0.000000e+00,Ethereum,0
LTC,0.000064,8.400000e-05,Litecoin,0
DASH,0.000009,2.200000e-05,Dash,1
XMR,0.000017,0.000000e+00,Monero,0
ETC,0.000115,2.100000e-04,Ethereum Classic,0
ZEC,0.000007,2.100000e-05,ZCash,0


In [65]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)